In [21]:
# load env
import os
from dotenv import load_dotenv
import json
import urllib
from pprint import pprint
load_dotenv()


# install dotenv
# pip install -U python-dotenv

True

In [2]:
# dummyepurl
url = os.environ['gpt2url']
# dummyepkey
api_key = os.environ['gpt2key']

In [6]:
textinput = 'I am a happy person'
deployment = 'none'

In [11]:
body = str(textinput)
payload = {
    "inputs": {
        "input_string": [body]
        }
    }

try: 
    payloads = str.encode(json.dumps(payload))
except Exception as e:
    print(e)

In [12]:
payloads

b'{"inputs": {"input_string": ["I am a happy person"]}}'

In [14]:
# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ api_key)}
if deployment != 'none':
    headers['azureml-model-deployment'] = deployment

req = urllib.request.Request(url, payloads, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    depslot = response.headers['azureml-model-deployment']

    print({'response': result, 'deployment': depslot})

except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))
    print(error.code)

{'response': b'[{"0": "I am a happy person, but I hope everyone is proud that you are a great person too\\". The comments are all very encouraging, even as you know they make me nervous. I am only 14 years old and I love to dance with my friends"}]', 'deployment': 'gpt2-3'}


In [22]:
text_completion_resp = json.loads(result.decode('utf-8'))

pprint(text_completion_resp)

[{'0': 'I am a happy person, but I hope everyone is proud that you are a great '
       'person too". The comments are all very encouraging, even as you know '
       'they make me nervous. I am only 14 years old and I love to dance with '
       'my friends'}]


In [33]:
# get swagger file 
url = "https://maggpt2.australiaeast.inference.ml.azure.com/swagger.json"

api_key = os.environ['gpt2key']
headers = {
    'Content-Type':'application/json',
    'Authorization':('Bearer '+ api_key)}
req = urllib.request.Request(url, None, headers)
response = urllib.request.urlopen(req)
result = response.read()
swagger = json.loads(result.decode('utf-8'))
pprint(swagger)


{'consumes': ['application/json'],
 'definitions': {'ErrorResponse': {'properties': {'message': {'type': 'string'},
                                                  'status_code': {'format': 'int32',
                                                                  'type': 'integer'}},
                                   'type': 'object'},
                 'ServiceInput': {'example': {}, 'type': 'object'},
                 'ServiceOutput': {'example': {}, 'type': 'object'}},
 'info': {'description': 'API specification for the Azure Machine Learning '
                         'service ML service',
          'title': 'ML service',
          'version': '1.0'},
 'paths': {'/': {'get': {'description': 'Simple health check endpoint to '
                                        'ensure the service is up at any given '
                                        'point.',
                         'operationId': 'ServiceHealthCheck',
                         'responses': {'200': {'description': 'If 

In [34]:
print(swagger)

{'swagger': '2.0', 'info': {'title': 'ML service', 'description': 'API specification for the Azure Machine Learning service ML service', 'version': '1.0'}, 'schemes': ['https'], 'consumes': ['application/json'], 'produces': ['application/json'], 'securityDefinitions': {'Bearer': {'type': 'apiKey', 'name': 'Authorization', 'in': 'header', 'description': 'For example: Bearer abc123'}}, 'paths': {'/': {'get': {'operationId': 'ServiceHealthCheck', 'description': 'Simple health check endpoint to ensure the service is up at any given point.', 'responses': {'200': {'description': "If service is up and running, this response will be returned with the content 'Healthy'", 'schema': {'type': 'string'}, 'examples': {'application/json': 'Healthy'}}, 'default': {'description': 'The service failed to execute due to an error.', 'schema': {'$ref': '#/definitions/ErrorResponse'}}}}}, '/score': {'post': {'operationId': 'RunMLService', 'description': "Run web service's model and get the prediction output"